In [1]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
import re
from multiprocessing import Pool, cpu_count
from IPython.display import display

In [2]:
path = "../raw/www.cibo360.it"
foods = [food for food in os.listdir(path) if food[0] != "."]
tables = [table.split(".csv")[0] for table in os.listdir("{path}/{food}".format(path=path, food=foods[0])) if table.endswith(".csv")]

In [3]:
tables

['minerali',
 'acidi_grassi',
 'vitamine',
 'composizione_chimica',
 'aminoacidi',
 'energia']

In [4]:
coefficents = {
    "g":1,
    "mg":1e-3,
    "µg":1e-6,
    "%":1
}

# Minerali parser

In [5]:
def parse_minerali(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="minerali")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    df /= 1000 # mg to g
    return df

# acidi_grassi

In [6]:
def parse_acidi_grassi(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="acidi_grassi")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    # already in g
    return df

# vitamine

In [7]:
def parse_vitamine(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="vitamine")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    units = df["2"].tolist()
    coeff = [coefficents[x] for x in units]
    df["1"] = df["1"] * coeff
    # result in g
    return df.iloc[:, :-1] # drop the units col

# composizione_chimica

In [8]:
def parse_composizione_chimica(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="composizione_chimica")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    units = df["2"].tolist()
    coeff = [coefficents[x] if pd.notna(x) else 1 for x in units]
    df["1"] = df["1"] * coeff
    df.drop("di cui", inplace=True)
    # result in g
    return df.iloc[:, :-1] # drop the units col

# aminoacidi

In [67]:
def parse_aminoacidi(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="aminoacidi")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="composizione_chimica")
    df2 = pd.read_csv(file_path)
    df2 = df2.set_index("0")
    
    parte_edibile = df2.loc["Parte edibile"][0]
    
    df = df.iloc[:, :-1] # drop the last col
    df.loc[d.index[:-2],["1"]] = df[d.index[:-2],["1"]].astype(float) * (parte_edibile/100)
    
    return df

In [68]:
d = parse_aminoacidi(foods[10])

TypeError: unhashable type: 'Index'

# energia

In [10]:
def parse_energia(food):
    file_path = "{path}/{name}/{table}.csv".format(path=path, name=food, table="energia")
    df = pd.read_csv(file_path)
    df = df.set_index("0")
    return df.head(1)

# function list

In [11]:
functions = [
    parse_minerali,
    parse_acidi_grassi,
    parse_vitamine,
    parse_composizione_chimica,
    parse_aminoacidi,
    parse_energia
]

In [18]:
def get_single_row(food):
    print(foods.index(food))
    csvs = [f(food) for f in functions]
    single_row = pd.concat(csvs).T
    single_row = single_row.rename({"1":food})
    single_row = single_row.iloc[:,~single_row.columns.duplicated()]
    del single_row.columns.name
    return single_row

In [19]:
with Pool(cpu_count()) as p:
    mpt = pd.concat(list(tqdm(p.imap(get_single_row, foods), total=len(foods))))

0
1
3
2


4
5
6
7
8
9
10


TypeError: Could not operate 1.0 with block values can't multiply sequence by non-int of type 'float'